In [11]:

import os as os
import gzip as gz

folder = '/TL/deep/fhgfs/projects/pebert/thesis/projects/statediff/references'

inputfile = os.path.join(folder, 'EnsRB_v78_hg38_multi.gff.gz')
outputfile = os.path.join(folder, 'EnsRB_v78_hg38_multi.bed')

feat_map = {'TF_binding_site': 'tfbs', 'enhancer': 'enh',
            'CTCF_binding_site': 'ctcf', 'open_chromatin_region': 'open',
            'promoter': 'prom', 'promoter_flanking_region': 'flank'}

buffer = []
with gz.open(inputfile, 'rt') as gff:
    for line in gff:
        if line:
            parts = line.split(maxsplit=8)
            chrom, start, end, regtype = parts[0], parts[3], parts[4], parts[2]
            if not chrom.startswith('chr'):
                continue
            regtype = feat_map[regtype]
            regid = 'noid'
            for attr in parts[8].split(';'):
                if attr.startswith('ID='):
                    regid = attr.split('=')[1]
                    break
            if regid == 'noid':
                raise ValueError('No ID: {}'.format(line.strip()))
            buffer.append((chrom, start, end, regid, str(int(end) - int(start)), '.', regtype))
        
buffer = sorted(buffer, key=lambda x: (x[0], int(x[1]), int(x[2])))

out_header = ['#chrom', 'start', 'end', 'name', 'length', 'strand', 'feature']
with open(outputfile, 'w') as dump:
    buffer = ['\t'.join(t) for t in buffer]
    buffer.append('')
    _ = dump.write('\t'.join(out_header))
    _ = dump.write('\n')
    _ = dump.write('\n'.join(buffer))
